In [7]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split

class LSTMNetwork(nn.Module):
    """A simple LSTM network for sequence prediction."""
    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMNetwork, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Return the last time step

In [2]:
def load_pamap2_data():
    df = pd.read_parquet('../data/PAMAP2/protocol.parquet')
    # the labels are on the 2º column
    X = df.drop('activityID', axis=1)
    y = df['activityID']
    return X, y

In [5]:

def objective_function(hidden_size, num_layers, learning_rate):
    """
    Objective function for Bayesian Optimization.

    This function initializes the LSTM network with the given hyperparameters,
    trains it on the provided dataloader, and returns the average loss as
    the evaluation metric.

    Parameters:
    -----------
    hidden_size : float
        The number of hidden units in the LSTM layer. This value should be
        between 10 and 100.

    num_layers : float
        The number of LSTM layers in the network. This value should be between
        1 and 5.

    learning_rate : float
        The learning rate for the optimizer. This value should be between
        0.0001 and 0.1.

    Returns:
    --------
    float
        The average loss of the model after training.
    """
    # Load PAMAP2 dataset
    X, y = load_pamap2_data()
    NUM_EPOCHS = 25
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Set up the LSTM network
    input_size = X.shape[1]  # Number of features
    output_size = 1          # Single output for binary classification

    model = LSTMNetwork(input_size, int(hidden_size), int(num_layers), output_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    model.train()  # Set model to training mode
    total_loss = 0
    
    dataloader = load_pamap2_data()

    for _ in range(NUM_EPOCHS):
        for x, y in dataloader:
            optimizer.zero_grad()
            output = model(x.float())
            loss = criterion(output, y.long())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    # Evaluation phase
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for x, y in dataloader:
            output = model(x.float())
            all_preds.append(output.softmax(dim=1).cpu().numpy())  # Convert to probabilities
            all_labels.append(y.cpu().numpy())

    # Concatenate results for AUC-ROC calculation
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Compute AUC-ROC score
    auc_score = roc_auc_score(all_labels, all_preds, multi_class='ovr')

    return auc_score  # Return the AUC-ROC score

In [6]:
pbounds = {
    'hidden_size': (10, 100),  # Example bounds for hidden layer size
    'num_layers': (1, 5),      # Example bounds for number of layers
    'learning_rate': (0.0001, 0.1)  # Example bounds for learning rate
}

# Run Bayesian Optimization
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=5,
    n_iter=25,
)

print(optimizer.max)

|   iter    |  target   | hidden... | learni... | num_la... |
-------------------------------------------------------------
0 of 25


ValueError: could not determine the shape of object type 'DataFrame'